In [54]:
import pandas as pd
import numpy as np
from natasha import MorphVocab, Doc, Segmenter, NamesExtractor
import gensim
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models import LdaModel, coherencemodel
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim_models
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)
import re
import pymorphy2
import string
import os
import shutil
import time
from multiprocessing import Process
from nltk.corpus import stopwords

In [2]:
t1 = pd.read_csv('datasets/ru_tweets/negative.csv', delimiter=';')
t2 = pd.read_csv('datasets/ru_tweets/positive.csv', delimiter=';') # , on_bad_lines='skip'

t1.columns.values[3] = 'text'
t1.columns.values[4] = 'label'
t2.columns.values[3] = 'text'
t2.columns.values[4] = 'label'

t1.shape, t2.shape

((111922, 12), (114910, 12))

In [3]:
df = pd.concat([t1, t2])

# Очистка текста

In [11]:
morph = pymorphy2.MorphAnalyzer()

In [4]:
def del_names(text):
    names = re.findall('[А-Я][а-я*]+', text)
    # print(names)
    filtered_names = []
    for i in names:
        type_ = str(morph.parse(i)[0].tag)
        # print(type_)
        for j in ['Name', 'Surn', 'Patr']:
            if j in type_:
                filtered_names.append(i)
                break
       

    # print(filtered_names)
    for i in filtered_names:
        text = text.replace(i, '')

    return text

In [12]:
del_names('лабораторию Чарльза Фрэнсиса Дженкинса, изобретателя,  в журнале Works in Progress.')

'лабораторию   , изобретателя,  в журнале Works in Progress.'

In [13]:
def lemmatizer(text):
    text = text.split()
    new_text = []

    for i in text:
        tmp = morph.parse(i)[0]
        new_text.append(tmp.normal_form)
    
    return ' '.join(new_text)

In [14]:
lemmatizer('не обращали внимания на алмазную промышленность')

'не обращать внимание на алмазный промышленность'

In [17]:
punctuation = string.punctuation + '«»\t—…’–№\xa0\n'

def replace_punctuation(text):
    for i in punctuation:
        text = text.replace(i, ' ')
    return text

In [1]:
def clear_text(row):
    text = row['text']

    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text) # Удаляем ссылки
    text = re.sub('@[^\s]+', 'USER', text)
    text = del_names(text) # Удаляем имена
    text = text.lower().replace("ё", "е") 
    text = replace_punctuation(text) # Удаляем пунктуацию

    symbols = re.findall("[A-Za-zА-Яа-я\s]", text) # Оставляем только буквы и пробельные символы
    text = ''.join(symbols)
    
    text = re.sub(' +', ' ', text) # Заменяем несколько пробелов на 1
    text = lemmatizer(text) # Приводим слова к нормальной форме
    row['text'] = text.strip()
    return row

In [21]:
# clear_text = df.apply(clear_text, axis=1)
# clear_text.shape

### Разбиваем обработку текстов между ядрами

In [22]:
dir_name = '.ds_tmp'

In [23]:
def devide_texts(df, parts = 5):
    l = len(df)//parts
    data = []
    amount = 0
    
    for i in range(parts):
        if i + 1 == parts:
            data.append(df[i*l:-1])
        else:
            data.append(df[i*l:(i+1)*l])
            
        # print(len(data[i]), i*l, (i+1)*l)
        amount+=len(data[i])
    
    print('Amount:', amount)
    for i, value in enumerate(data, start = 1):
       print(f'Part {i}, data len: {len(value)}')

    return data

In [ ]:
data = devide_texts(df)

In [25]:
def worker(i, data):
    text = data.apply(clear_text, axis=1)
    text.to_csv(f'{dir_name}/clean_text_{i}.csv')

In [27]:
def join_results(parts = 5):
    data = []

    for i in range(parts):
        data.append(pd.read_csv(f'{dir_name}/clean_text_{i}.csv', index_col=0))
    
    return pd.concat(data)

In [33]:
def preprocess(data, num_workers = 5, ds_name='clean_text.csv'):
    thread_list = []

    os.mkdir(dir_name)
    start_time = time.time()

    for i in range(num_workers):
        thread_list.append(Process(target=worker, args=(i, data[i])))
        thread_list[i].start()

    for i in range(num_workers):
        thread_list[i].join()

    df = join_results()
    df.to_csv(ds_name)
    
    end_time = time.time()
    print(f'Elapsed time: {end_time-start_time} sec')
    print(f'Saved to {ds_name}')
    shutil.rmtree(dir_name)

In [2]:
preprocess(data, ds_name='twitter_clean_text_theme.csv')

# Load clean text

In [41]:
df = pd.read_csv('datasets/twitter_clean_text_theme.csv', index_col=0)
df[['text', 'label']]

/tmp/ipykernel_29175/3457987990.py:1: DtypeWarning: Columns (3,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('datasets/twitter_clean_text_theme.csv', index_col=0)


,text,label
0,коллега сидеть рубиться в urban terror а я из ...,-1
1,user как говорить обещаной три год ждать,-1
2,желать хороший полёт и удачный посадка я быть ...,-1
3,обновить за какой то леший surf теперь не рабо...,-1
4,котёнок вчера носик разбить плакать и расстраи...,-1
...,...,...
114904,торт с чернослив купить на кропоток в тот мага...,1
114905,спасть в родительский дом на свой кровать прос...,1
114906,rt user эх мы немного решить сократить путь се...,1
114907,что происходить с я когда в эфир proactivefm з...,1


In [55]:
stop_list = stopwords.words("russian")
stop_list.extend(['это', 'который', 'всё', 'весь', 'свой', 'наш', 'ещё', 'также', 'год', 'c', 'е', 'с', 'e', 'b', 'a', 'gb', 'd'])

In [56]:
stop_list

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [57]:
tokenized_text = []

for i in df.text:
    tokenized_text.append([j for j in i.split() if j not in stop_list])

tokenized_text[0][:10], len(tokenized_text)

(['коллега',
  'сидеть',
  'рубиться',
  'urban',
  'terror',
  'долбать',
  'винд',
  'мочь'],
 226831)

In [58]:
bigram = Phrases(tokenized_text, min_count=10)

In [59]:
docs = [bigram[i] for i in tokenized_text]

for i in docs:
    tmp = [j for j in i if '_' in j]

    if len(tmp) > 10:
        break

tmp

['sm_amp',
 'gt_напомнить',
 'называться_фотка',
 'вокруг_чёрный',
 'рамка_внизу',
 'надпись_au',
 'amp_gt',
 'некролог_sm',
 'amp_gt',
 'au_amp',
 'gt_демотиватор',
 'sm_amp']

In [60]:
dictionary = Dictionary(docs)
dictionary.filter_extremes(no_below=10, no_above=0.3)

In [61]:
id2token = {value: key for key, value in dictionary.token2id.items()}

In [62]:
corpus = [dictionary.doc2bow(i) for i in docs] # bag-of-words

In [63]:
print('Unique tokens:', len(dictionary))
print('Amount of docs:', len(corpus))

Unique tokens: 13464
Amount of docs: 226831


In [68]:
# Set training parameters.
num_topics = 6
chunksize = 4000
passes = 20
iterations = 400
eval_every = None 

In [69]:
%%time

id2word = id2token 

model = gensim.models.ldamulticore.LdaMulticore(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    workers=16
)

CPU times: user 58.2 s, sys: 4.44 s, total: 1min 2s
Wall time: 2min 30s


In [70]:
# Coherence (чем ближе к нулю, тем лучше 0 the better)
coherence_model = coherencemodel.CoherenceModel(model=model, texts=docs, dictionary=dictionary, coherence='u_mass')
coherence_score = coherence_model.get_coherence()
print(f'Coherence Score: {coherence_score:.4f}')

Coherence Score: -4.4801


In [71]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
pyLDAvis.display(vis)

In [72]:
model.print_topics()

[(0,
  '0.137*"rt" + 0.104*"url" + 0.017*"спасибо" + 0.014*"хотеть" + 0.009*"хороший" + 0.008*"любимый" + 0.008*"правда" + 0.006*"привет" + 0.006*"день_рождение" + 0.005*"очень"'),
 (1,
  '0.019*"делать" + 0.016*"rt" + 0.010*"хотеть" + 0.010*"сделать" + 0.010*"завтра" + 0.010*"хороший" + 0.009*"пойти" + 0.009*"спать" + 0.009*"написать" + 0.009*"url"'),
 (2,
  '0.026*"новый" + 0.022*"rt" + 0.019*"url" + 0.017*"время" + 0.014*"просто" + 0.011*"очень" + 0.007*"играть" + 0.006*"купить" + 0.006*"мочь" + 0.006*"песня"'),
 (3,
  '0.017*"rt" + 0.011*"мочь" + 0.011*"большой" + 0.010*"хотеться" + 0.009*"думать" + 0.008*"человек" + 0.007*"хотеть" + 0.007*"очень" + 0.007*"жизнь" + 0.007*"сегодня"'),
 (4,
  '0.027*"url" + 0.026*"знать" + 0.024*"любить" + 0.014*"человек" + 0.013*"говорить" + 0.012*"почему" + 0.011*"просто" + 0.010*"мочь" + 0.010*"вообще" + 0.007*"сказать"'),
 (5,
  '0.025*"сегодня" + 0.024*"день" + 0.015*"хотеть" + 0.011*"дом" + 0.009*"идти" + 0.009*"завтра" + 0.009*"школа" + 0.007*

In [73]:
print(model.alpha) # Document-topic prior
print(model.eta) # Topic-word prior

[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
[0.33690485 0.29532507 0.40619922 ... 0.11745608 0.11745608 0.11745608]


In [74]:
topics = list(model.get_document_topics(corpus, minimum_probability=0))
print(len(topics))
print(topics[0][:30])
print(topics[1][:30])

print()
total = 0
for i in range(num_topics):
    tmp = sum([pair[i][1] for pair in topics])/len(topics)
    print(f"Topic {i} proportion: {100*tmp:.1f}%")
    total+=tmp

print(f'Total: {total * 100:.2f}%')

226831
[(0, 0.023905441), (1, 0.024040177), (2, 0.88011026), (3, 0.023987656), (4, 0.023950985), (5, 0.024005454)]
[(0, 0.05618668), (1, 0.055569578), (2, 0.056246325), (3, 0.055600457), (4, 0.71920335), (5, 0.057193596)]

Topic 0 proportion: 16.5%
Topic 1 proportion: 15.4%
Topic 2 proportion: 14.3%
Topic 3 proportion: 16.3%
Topic 4 proportion: 18.0%
Topic 5 proportion: 19.6%
Total: 100.00%


In [75]:
def get_topic_num(topic):
    idx = 0
    for value in topic:
        if topic[idx][1] < value[1]:
            idx = value[0]
    return idx

In [76]:
get_topic_num(topics[0])

2

In [77]:
idx = 0

def set_topic(row):
    global idx
    row['topic'] = get_topic_num(topics[idx])
    idx+=1
    return row

df2 = df.apply(set_topic, axis=1)

In [79]:
df2[['text', 'topic']]

,text,topic
0,коллега сидеть рубиться в urban terror а я из ...,2
1,user как говорить обещаной три год ждать,4
2,желать хороший полёт и удачный посадка я быть ...,0
3,обновить за какой то леший surf теперь не рабо...,0
4,котёнок вчера носик разбить плакать и расстраи...,5
...,...,...
114904,торт с чернослив купить на кропоток в тот мага...,2
114905,спасть в родительский дом на свой кровать прос...,5
114906,rt user эх мы немного решить сократить путь се...,3
114907,что происходить с я когда в эфир proactivefm з...,4


In [82]:
x1 = [1, 2, 3, 4, 5]
np.random.shuffle(x1)
x1

[3, 5, 2, 4, 1]

In [89]:
y = df2[df2['topic'] == 0]['text'].to_list()
np.random.shuffle(y)
len(y[:3000]) # get 15 000 comments, 15/5 = 3

3000

In [88]:
26003 - (10552)

15451

In [90]:
15000/226831

0.06612852740586604

In [3]:
df_origin = pd.concat([t1, t2])
df_origin.shape

In [96]:
df_origin

,408906762813579264,1386325944,dugarchikbellko,text,label,0,0.1,0.2,8064,111,94,2,408906692374446080,1386325927,pleease_shut_up,7569,62,61,0.3
0,4.089068e+17,1.386326e+09,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26.0,42.0,39.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.089069e+17,1.386326e+09,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718.0,49.0,249.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.089069e+17,1.386326e+09,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628.0,207.0,200.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.089069e+17,1.386326e+09,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35.0,17.0,34.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.089069e+17,1.386326e+09,nunejibaduq,"Котёнка вчера носик разбила, плакала и расстра...",-1,0,0,0,222.0,62.0,62.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114905,NaN,NaN,NaN,"Спала в родительском доме, на своей кровати......",1,0,0,0,NaN,NaN,NaN,NaN,4.113687e+17,1.386913e+09,diminlisenok,1497.0,56.0,34.0,2.0
114906,NaN,NaN,NaN,RT @jebesilofyt: Эх... Мы немного решили сокра...,1,0,1,0,NaN,NaN,NaN,NaN,4.113687e+17,1.386913e+09,qilepocagotu,692.0,225.0,210.0,0.0
114907,NaN,NaN,NaN,"Что происходит со мной, когда в эфире #proacti...",1,0,0,0,NaN,NaN,NaN,NaN,4.113688e+17,1.386913e+09,DennyChooo,4905.0,448.0,193.0,13.0
114908,NaN,NaN,NaN,"""Любимая,я подарю тебе эту звезду..."" Имя како...",1,0,0,0,NaN,NaN,NaN,NaN,4.113688e+17,1.386913e+09,bedowabymir,989.0,254.0,251.0,0.0


In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226831 entries, 0 to 114908
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   408906762813579264  111922 non-null  float64
 1   1386325944          111922 non-null  float64
 2   dugarchikbellko     111922 non-null  object 
 3   text                226831 non-null  object 
 4   label               226831 non-null  int64  
 5   0                   226831 non-null  int64  
 6   0.1                 226831 non-null  int64  
 7   0.2                 226831 non-null  int64  
 8   8064                111922 non-null  float64
 9   111                 111922 non-null  float64
 10  94                  111922 non-null  float64
 11  2                   111922 non-null  float64
 12  408906692374446080  114909 non-null  float64
 13  1386325927          114909 non-null  float64
 14  pleease_shut_up     114909 non-null  object 
 15  7569                114909 non-null  fl

In [93]:
df.shape, df2.shape

((226831, 19), (226831, 20))

In [98]:
df2

,408906762813579264,1386325944,dugarchikbellko,text,label,0,0.1,0.2,8064,111,94,2,408906692374446080,1386325927,pleease_shut_up,7569,62,61,0.3,topic
0,4.089068e+17,1.386326e+09,nugemycejela,коллега сидеть рубиться в urban terror а я из ...,-1,0,0,0,26.0,42.0,39.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,4.089069e+17,1.386326e+09,4post21,user как говорить обещаной три год ждать,-1,0,0,0,718.0,49.0,249.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
2,4.089069e+17,1.386326e+09,Poliwake,желать хороший полёт и удачный посадка я быть ...,-1,0,0,0,10628.0,207.0,200.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,4.089069e+17,1.386326e+09,capyvixowe,обновить за какой то леший surf теперь не рабо...,-1,0,0,0,35.0,17.0,34.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,4.089069e+17,1.386326e+09,nunejibaduq,котёнок вчера носик разбить плакать и расстраи...,-1,0,0,0,222.0,62.0,62.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114904,NaN,NaN,NaN,торт с чернослив купить на кропоток в тот мага...,1,0,0,0,NaN,NaN,NaN,NaN,4.113687e+17,1.386913e+09,see_all_sea,9787.0,49.0,32.0,0.0,2
114905,NaN,NaN,NaN,спасть в родительский дом на свой кровать прос...,1,0,0,0,NaN,NaN,NaN,NaN,4.113687e+17,1.386913e+09,diminlisenok,1497.0,56.0,34.0,2.0,5
114906,NaN,NaN,NaN,rt user эх мы немного решить сократить путь се...,1,0,1,0,NaN,NaN,NaN,NaN,4.113687e+17,1.386913e+09,qilepocagotu,692.0,225.0,210.0,0.0,3
114907,NaN,NaN,NaN,что происходить с я когда в эфир proactivefm з...,1,0,0,0,NaN,NaN,NaN,NaN,4.113688e+17,1.386913e+09,DennyChooo,4905.0,448.0,193.0,13.0,4


In [105]:
len(df2['408906762813579264'].unique())+ len(df2['408906692374446080'].unique())

226816

In [107]:
len(df_origin['408906762813579264'].unique()) + len(df_origin['408906692374446080'].unique())

226817

In [4]:
# Так как мы предобработали текст и не сохранили его в первоначальном виде,
# то объединяем исходные данные с получившимся данными и 
# сохраняем из каждой темы N строк в неформатированном виде (без лемматизации, удаления стоп слов и т.д.)

In [109]:
def create_id(row):
    if not np.isnan(row['408906762813579264']):
        row['id'] = row['408906762813579264']
    elif not np.isnan(row['408906692374446080']):
        row['id'] = row['408906692374446080']
    return row

# df_origin[:10].apply(create_id, axis=1)

In [110]:
theme_splited = df2.apply(create_id, axis=1)
df_origin =  df_origin.apply(create_id, axis=1)

In [111]:
merged = pd.merge(theme_splited, df_origin, on='id', how='left')

In [113]:
merged[['text_x', 'text_y', 'label_x', 'label_y', 'topic', '']]

,text_x,text_y,label_x,label_y,topic
0,коллега сидеть рубиться в urban terror а я из ...,NaN,-1,NaN,2
1,user как говорить обещаной три год ждать,@elina_4post как говорят обещаного три года жд...,-1,-1.0,4
2,желать хороший полёт и удачный посадка я быть ...,"Желаю хорошего полёта и удачной посадки,я буду...",-1,-1.0,0
3,обновить за какой то леший surf теперь не рабо...,"Обновил за каким-то лешим surf, теперь не рабо...",-1,-1.0,0
4,котёнок вчера носик разбить плакать и расстраи...,"Котёнка вчера носик разбила, плакала и расстра...",-1,-1.0,5
...,...,...,...,...,...
226856,торт с чернослив купить на кропоток в тот мага...,Торт с черносливом :) купленный на кропотке в ...,1,1.0,2
226857,спасть в родительский дом на свой кровать прос...,"Спала в родительском доме, на своей кровати......",1,1.0,5
226858,rt user эх мы немного решить сократить путь се...,RT @jebesilofyt: Эх... Мы немного решили сокра...,1,1.0,3
226859,что происходить с я когда в эфир proactivefm з...,"Что происходит со мной, когда в эфире #proacti...",1,1.0,4


In [140]:
def get_n_rows_by_topic(df, topic_num, amount):
    topic = df[df['topic'] == topic_num]
    return topic.sample(n = amount)

In [129]:
required_cols = merged[['text_y', 'label_y', 'topic']].dropna()

In [136]:
# get_n_rows_by_topic(required_cols, 4, 300)

In [141]:
result = pd.concat([get_n_rows_by_topic(required_cols, i, 3000) for i in range(5)])
result.shape

(15000, 3)

In [123]:
y = merged[merged['topic'] == 0][['text_y', 'label_y', 'topic']].dropna(subset=['text_y', 'label_y'])
y.sample(n=3000)
# np.random.shuffle(y)

,text_y,label_y,topic
200236,Мазафака)))он даже и не к казахам заежает))бра...,1.0,0
77364,RT @tutby: 10 тысяч долларов за первого ребенк...,-1.0,0
51165,ох как мне это знакомо!(( http://t.co/cibN5nVFjP,-1.0,0
176293,"Хочу большую и пушистую ёлку, такую, как была ...",1.0,0
182645,Отличный вечер**\nСпасибо вам:))))\nТолько Юль...,1.0,0
...,...,...,...
93859,RT @sladkaya_e: Как же я не хочу завтра в школу(,-1.0,0
126959,RT @_8763731544813: Насобирал грибочков летом....,1.0,0
120933,"RT @STONED_PONY: - Что это за клей, который де...",1.0,0
158640,"RT @BOPOBKA_of_LOVE: И не говори, что не пропе...",1.0,0


In [142]:
result

,text_y,label_y,topic
181683,@BananaStyles19 @2013Viks @nastenka265 @ANovic...,1.0,0
153236,Q: Привет:-) Ты с Улан-Удэ? A: http://t.co/...,1.0,0
200680,Концерт woodkid однозначно в топе лучших! Спас...,1.0,0
209768,"RT @Margaret_Kwon: @1djulialove ни стыда, ни с...",1.0,0
168823,"RT @alexpike13: Да, это правда - #Орейро в сво...",1.0,0
...,...,...,...
14751,"Надо было DD-WRT ему шить, чтобы не заябылвал. :(",-1.0,4
112481,Задайте мне любой вопрос на trislovamne .ru :)...,1.0,4
40314,"Все бабы стервы, и мир бардак! И друг, к тому ...",-1.0,4
190736,"ах вспомнила ""Вик, нужно меняться"" ""Ты хотя бы...",1.0,4


In [144]:
x = result.groupby(by='topic').count()
print(x)

       text_y  label_y
topic                 
0        3000     3000
1        3000     3000
2        3000     3000
3        3000     3000
4        3000     3000


In [145]:
result.to_csv('datasets/twitter.csv')

In [147]:
result.dropna(subset=['label_y']).shape

(15000, 3)

In [148]:
result['label_y'].unique()

array([ 1., -1.])

In [149]:
new_df = pd.read_csv('datasets/twitter.csv', index_col=0)
new_df.shape, new_df['label_y'].unique()

((15000, 3), array([ 1., -1.]))